In [1]:
#mm

In [2]:
from mmilanutil import *

In [3]:
mmptmaxabs

<function mmilanutil.mmptmaxabs(x)>

In [2]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("d:/ai/myvocab/runs")

In [3]:
writer.flush()


<a href="https://colab.research.google.com/github/Taaniya/exploring-gpt2-language-model/blob/main/Visualizing_gpt2_token_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook visualizes word token embeddings of GPT2 on Tensorboard projector

In [6]:
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
#! pip install transformers

In [6]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
from tensorboard.plugins import projector

import re
import os
from tqdm import tqdm

In [7]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [9]:
word_embeddings = model.transformer.wte.weight      # Word Token Embeddings 
position_embeddings = model.transformer.wpe.weight  # Word Position Embeddings 

NameError: name 'model' is not defined

In [8]:
print(word_embeddings.shape)

print(position_embeddings.shape)

NameError: name 'word_embeddings' is not defined

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

In [ ]:
tokenizer.pretrained_vocab_files_map

In [41]:
tokenizer.vocab

{'Ġpayment': 6074,
 'Ġ<+': 42496,
 'ĠOct': 2556,
 'Ġwrest': 10419,
 'Ġpath': 3108,
 'Ġsupplying': 28099,
 'ĠMile': 26004,
 'ĠShak': 35274,
 'Ġharb': 43560,
 'ĠGlow': 45929,
 'Ġdecad': 46734,
 'ĠHades': 36857,
 'Ġnighttime': 45324,
 'ODUCT': 28644,
 'ĠKasich': 24931,
 'Ġsubsequent': 8840,
 'Ġanomalies': 35907,
 'rupulous': 46272,
 'ard': 446,
 'active': 5275,
 'Ġscrutiny': 14521,
 'idget': 17484,
 ":'": 32105,
 'Ġgranite': 41013,
 'Transfer': 43260,
 'Ġunusually': 23708,
 'Sync': 28985,
 'ial': 498,
 'Ġmelodies': 47077,
 'Sel': 48767,
 'Ġdeploying': 29682,
 'eno': 23397,
 'ĠHart': 11345,
 'ĠHere': 3423,
 'Attack': 27732,
 'hospital': 49257,
 'Ġoverwrite': 49312,
 'lando': 11993,
 'day': 820,
 'Ġfren': 22832,
 'ĠdÃ©': 39073,
 'GES': 48075,
 'ĠTal': 7193,
 'Ġlibrary': 5888,
 'ennes': 42573,
 'achers': 17892,
 'Ġhon': 3032,
 'ĠString': 10903,
 'artney': 41709,
 'dist': 17080,
 'isphere': 22833,
 'Ġroads': 9725,
 'Ġthinly': 44722,
 'Ġconcede': 33760,
 'Ġlocation': 4067,
 'Ġfingert': 35676,


**Creating list of tokens in vocab sorted by their index in vocab**

In [42]:
vocab_list = sorted(tokenizer.vocab.items(), key=lambda x:x[1])

**Verify if the resulting list is sorted by the token indices.**

In [43]:
for k,v in tokenizer.vocab.items():
    if v < 10:
        print(k, v)

* 9
% 4
# 2
" 1
! 0
' 6
$ 3
( 7
& 5
) 8


In [44]:
vocab_list[:10]

[('!', 0),
 ('"', 1),
 ('#', 2),
 ('$', 3),
 ('%', 4),
 ('&', 5),
 ("'", 6),
 ('(', 7),
 (')', 8),
 ('*', 9)]

**Save the sorted token labels from vocab as metadata file**

In [45]:
# Just create the metadata file

with open(os.path.join(log_dir, 'metadata.tsv'), "w") as f:
    
    #aa tqdm
    for word, idx in tqdm(vocab_list):
        line = str(word.encode(encoding='iso-8859-1', errors='replace'))
        line = re.sub("^b'", "", line)
        line = re.sub('^b"', "", line)
        line = re.sub("'$", "", line)
        line = re.sub('"$', '', line)
        f.write("{}\n".format(line))

100%|██████████████████████████████████| 50257/50257 [00:00<00:00, 58295.29it/s]


**Save the word embeddings**

In [46]:
writer.add_embedding(model.transformer.wte.weight.detach().numpy())

AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'get_filesystem'

In [16]:
embeddings = tf.Variable(model.transformer.wte.weight.detach().numpy())
checkpoint = tf.train.Checkpoint(embedding=embeddings)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

'd:/ai/logs/vocab/embedding.ckpt-1'

Finally set up tensorboard projector's configuration. This creates a configuration file with .pbtxt extension.

In [20]:
# Set up config.
config = projector.ProjectorConfig()
embedding = config.embeddings.add()

# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

Run tensorboard to visualize the embeddings. Use UMAP for faster and cleaner visualizations. Search a few keywords and find their nearest neighbours in the 3D space and in the drop down.

In [ ]:
writer.flush()


In [ ]:
#mm

%load_ext tensorboard
%tensorboard --logdir d:/ai

#### References

1. [Tensorboard embedding projector](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin)

2. https://towardsdatascience.com/how-to-visualize-text-embeddings-with-tensorboard-47e07e3a12fb

3. https://github.com/huggingface/transformers/issues/1458